In [25]:
import re
import csv
import pandas as pd
import numpy as np
from typing import Pattern

from textblob.classifiers import NaiveBayesClassifier
import nltk 
from nltk.stem.snowball import FrenchStemmer 
from nltk.corpus import stopwords 
stop_words = set(stopwords.words('french'))
from sklearn.preprocessing import LabelEncoder


In [32]:
path_init = "data/_57793TransmissionsSurvivors_2014V1.2.csv"
path_train = 'data/TransmissionsSurvivors_2014V1.100-sentiments.csv'
path_new_sentiments = "data/no-clean-text-sentiments.csv"

PATTERN_SUPP_HTML : Pattern = re.compile(r'<(.*?)>')
REPLACE_NO_SPACE : Pattern = re.compile(r"['.;€:!*=\%?,<>\"()\[\]]")
REPLACE_WITH_SPACE : Pattern = re.compile(r"(<br\s*/><br\s*/>)|(\-)|(\/)|°")

In [33]:
# nettoyer le texte 
def preprocess_clean(reviews):
    #if type(reviews) != float:
    reviews = str(reviews).lower() # mettre le texte en minuscule

    reviews = re.sub(r'\bagrave\b', 'a', reviews) #replacer agrave par a (!à)
    reviews = re.sub(r'\beacut\b|\begrave\b|\bpreacute\b', 'e', reviews)#

    sub_html = re.findall(PATTERN_SUPP_HTML, reviews) #supprimer les balise html
    for html in sub_html:
        if html:
            reviews = re.sub(r'\s+', ' ', reviews)
            reviews = re.sub(r"\'", ' ', reviews)
            reviews = reviews.replace(html, ' ')
            reviews = reviews.replace('< >', ' ')
    
    reviews = REPLACE_NO_SPACE.sub(' ', reviews) #supprimer tout les caracteres [.;:!=\%?,\"()\[\]]
    reviews = REPLACE_WITH_SPACE.sub(' ', reviews) #supprimer espace entre balise html au cas où y'en a 
    reviews = reviews.replace('ttt', 'traitement') 
    reviews = re.sub(r"([+])", ' plus ', reviews)
    reviews = re.sub(r"([+])", ' ', reviews)
    reviews = re.sub(r'[0-9]', ' ', reviews)
    #reviews = re.sub(r'\b\w{1,3}\b',' ', reviews)

    #La première manipulation souvent effectuée dans le traitement du texte est la suppression de ce qu'on 
    #appelle en anglais les stopwords. Ce sont les mots très courants dans la langue étudiée 
    #("et", "à", "le"... en français) qui n'apportent pas de valeur informative pour la compréhension 
    #du "sens" d'un document et corpus. Il sont très fréquents et ralentissent notre travail : 
    #nous souhaitons donc les supprimer.
    for word in reviews.split() :
        if word in stop_words or len(word) < 2:
            word = r'\b' + word + r'\b'
            reviews = re.sub(word, '', str(reviews))
         
    reviews = re.sub(' +', ' ', reviews)#supprimer tout espace de plus qui peut exister à la fin de traitement
    #print(reviews)
    return reviews

In [37]:
data = pd.read_csv(path_train, sep=";")
data.descriptions = data.descriptions.apply(lambda x: preprocess_clean(x))
data = data.sample(frac=1).reset_index(drop=True)

data.head(10)

,id_resident,id_etablissement,date_T,descriptions,sentiments
0,166,207,1418539640,selles plus plus plus,Negative
1,166,207,1418539640,filles donné laxatif suppositoire,Negative
2,192,207,1420031823,surveillance,Negative
3,192,207,1419660167,mr martini tousse depuis matin prévenir ide,Negative
4,208,207,1418917919,mme retour hospitalisation,Positive
5,213,207,1416896022,mal dormie acause mme miranda diambule ds coul...,Negative
6,192,207,1419773359,arrivée matin trouvé toilettes puis rendu sal...,Negative
7,169,207,1417346152,douloureuse pouce pied gauche,Negative
8,181,207,1418916482,mme refuser prendre gouter,Negative
9,208,207,1417625093,agitée plus plus plus plus jour refus alimente...,Negative


In [40]:
# classifier les transmissions survivant 2014 
def Classifier_sentiment(path):
    global classifier
    with open(path, 'r') as csvfile:
        train_data, X_train, test_data = [], [], []

        for _,_,_,descriptions, sentiments in  csv.reader(csvfile, delimiter=';'):
            if descriptions:
                descriptions = preprocess_clean(descriptions)#clean text 
                stopword_list = get_stopswords()
                filtered_word = filter_stopwords(descriptions, stopword_list)
                # print(filtered_word)
                descriptions = ' '.join(filtered_word)
                data_tmp = descriptions, sentiments
                #print(descriptions) 
                train_data.append(data_tmp)

        # split le dataset en train et test 
        r = .0 
        for i in train_data:
            if r <= 8.5:
                X_train.append(i)
            else:
                test_data.append(i)

            r += .1

        # print(X_train)
        # classifier avec NaiveBayesClassifier de package textblob
        classifier = NaiveBayesClassifier(X_train)
        print(classifier.accuracy(test_data))

In [41]:
Classifier_sentiment(path_train)

0.6428571428571429


In [42]:
#Mettre les prédiction dans csv 
def classifier_to_csv(path, path_s):
	with open(path_s,  'w') as file_csv: # nouveau fichier csv avec les sentiment 	 
		with open(path, 'r') as csvfile: # csv à classifier
			spamwriter = csv.writer(file_csv, delimiter=';', quotechar='"', quoting=csv.QUOTE_MINIMAL)
			for rows in  csv.reader(csvfile, delimiter=';'):
				if rows:
					descriptions = rows[3]
					sentiment = classifier.classify(descriptions) # classifier descriptions
					spamwriter.writerow([rows[0], rows[1], rows[2], descriptions, sentiment])

In [44]:
!pwd

/Users/sabbar/Desktop/Korian/projects/analyse-sentiment
